# CatBoost

CatBoostは、Yandexの研究者とエンジニアによって開発された勾配ブースティングアルゴリズムを利用した決定木のためのライブラリです​​。このライブラリは2017年にリリースされ、ロシアの検索エンジン企業であるYandex社の検索アルゴリズムにも利用されています​​。CatBoostは、特にカテゴリカルデータの処理に優れ、高速なGPUトレーニング、およびモデルおよび特徴分析のための視覚化ツールも提供されている。

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/develop/tabledata/binary_classification/CatBoost.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install catboost

In [5]:
from catboost import CatBoost, Pool

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_breast_cancer

### データの用意

In [3]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
x = df.drop('target', axis = 1)
y = df['target']

X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

### CatBoostの学習

In [6]:
train_pool = Pool(X_train, y_train)
valid_pool = Pool(X_valid, y_valid)

# モデルを作成し訓練
model = CatBoost({'iterations': 500, 'learning_rate': 0.1, 'depth': 6, 'loss_function': 'MultiClass', 'verbose': 200})
model.fit(train_pool)


0:	learn: 0.6283834	total: 57.6ms	remaining: 28.7s
200:	learn: 0.0144338	total: 509ms	remaining: 757ms
400:	learn: 0.0063685	total: 1.02s	remaining: 251ms
499:	learn: 0.0050509	total: 1.28s	remaining: 0us


### 予測

In [10]:
y_pred = model.predict(valid_pool, prediction_type='Probability')

In [13]:
auc = roc_auc_score(y_valid, y_pred[:, 1])
print(f'AUC: {auc}')

AUC: 0.9954143465443825
